In [2]:
from pop import LiDAR, Pilot
from pop.Util import imshow
import time
from pop import Util
import cv2

# Enable image display using Util module
Util.enable_imshow()

lidar = LiDAR.Rplidar()
car = Pilot.AutoCar()


In [3]:
lidar.connect()
lidar.startMotor()

car.setSpeed(90)
car.steering = 0

In [ ]:


cam=Util.gstrmer(width=640, height=480, fps=60, flip=0)
Car = Pilot.AutoCar()
camera=cv2.VideoCapture(cam,cv2.CAP_GSTREAMER) #initialiser capture video
if not camera.isOpened():
    print("not found")
while True:
    # Read a frame from the camera
    ret, frame = camera.read()

    # Check if frame is read successfully
    if not ret:
        break
        
        
    no_collision = True

    vectors = lidar.getVectors()
    
    lidar_map = lidar.getMap(limit_distance = 2000, size=(50,50))
    imshow("map", lidar_map)
    
    filtered_vectors = []
    for degree, distance, *other_data in vectors:  # Use unpacking with *other_data
        if distance > 200:
            filtered_vectors.append((degree, distance))  # Only append relevant data


    # Check for obstacles on the left and right sides
    left_obstacle = min(distance for degree, distance in filtered_vectors if 210 <= degree <= 330)  # Left side range
    right_obstacle = min(distance for degree, distance in filtered_vectors if 30 <= degree <= 150)  # Right side range

    # Update steering based on obstacle detection
    if left_obstacle and left_obstacle < 700:
        car.steering = 1  # Turn right to avoid left obstacle
        car.setSpeed(60)  # Reduce speed while turning
    elif right_obstacle and right_obstacle < 700:
        car.steering = -1  # Turn left to avoid right obstacle
        car.setSpeed(60)  # Reduce speed while turning
    else:
        car.steering = 0  # No obstacle, move straight
        car.setSpeed(60)  # Higher speed for straight path

    car.forward()
    # Display the frame in a window named "v_test"
    cv2.imshow("v_test", frame)
  # Optional: Visualize LiDAR map (uncomment if needed)
  # lidar_map = lidar.getMap(limit_distance=2000, size=(50, 50))
  # imshow("map", lidar_map)

    time.sleep(0.1)  # Short delay between readings


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [ ]:
car.stop()
lidar.stopMotor()